<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkRP0101ENCoursera23911160-2022-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/M1_R_Basics/images/IDSNlogo.png" width="200" align="center">
</a>


<h1>Analysis of Global COVID-19 Pandemic Data</h1>

Estimated time needed: **90** minutes

**NOTE: This analysis has been done as a final project for edX course "R Programming Basics for Data Science"**

## Overview:

There are 10 tasks in this final project. All tasks will be graded by your peers who are also completing this assignment within the same session.

You need to submit the following the screenshot for the code and output for each task for review.

If you need to refresh your memories about specific coding details, you may refer to previous hands-on labs for code examples.


In [145]:
require("httr")
require("rvest")

library(httr)
library(rvest)
library(tidyr)
library(dplyr)

Note: if you can import above libraries, please use install.packages() to install them first.


## TASK 1: Get a `COVID-19 pandemic` Wiki page using HTTP request


First, let's write a function to use HTTP request to get a public COVID-19 Wiki page.

Before you write the function, you can open this public page from this

URL [https://en.wikipedia.org/w/index.php?title=Template\:COVID-19\_testing_by_country](https://en.wikipedia.org/w/index.php?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkRP0101ENCoursera23911160-2022-01-01&title=Template%3ACOVID-19\_testing_by_country) using a web browser.

The goal of task 1 is to get the html page using HTTP request (`httr` library)


In [120]:
get_wiki_covid19_page <- function() {
    
  # Our target COVID-19 wiki page URL is: https://en.wikipedia.org/w/index.php?title=Template:COVID-19_testing_by_country  
  # Which has two parts: 
    # 1) base URL `https://en.wikipedia.org/w/index.php  
    # 2) URL parameter: `title=Template:COVID-19_testing_by_country`, seperated by question mark ?
    
  # Wiki page base
  wiki_base_url <- "https://en.wikipedia.org/w/index.php"
  # You will need to create a List which has an element called `title` to specify which page you want to get from Wiki
  # in our case, it will be `Template:COVID-19_testing_by_country`
  query_params <- list(title = "Template:COVID-19_testing_by_country")
  # - Use the `GET` function in httr library with a `url` argument and a `query` argument to get a HTTP response
  response <- GET(wiki_base_url, query = query_params)
  # Use the `return` function to return the response
  return(response)
}

Call the `get_wiki_covid19_page` function to get a http response with the target html page


In [121]:
# Call the get_wiki_covid19_page function and print the response
get_wiki_covid19_page()

## TASK 2: Extract COVID-19 testing data table from the wiki HTML page


On the COVID-19 testing wiki page, you should see a data table `<table>` node contains COVID-19 testing data by country on the page:

<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkRP0101ENCoursera23911160-2022-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/M5_Final/images/covid-19-by-country.png" width="400" align="center">
</a>

Note the numbers you actually see on your page may be different from above because it is still an on-going pandemic when creating this notebook.

The goal of task 2 is to extract above data table and convert it into a data frame


Now use the `read_html` function in rvest library to get the root html node from response


In [122]:
# Get the root html node from the http response in task 1
root_node <- read_html(get_wiki_covid19_page())
root_node

Get the first table in the HTML root node using `html_node` function


In [123]:
# Get the table node from the root html node
table_node <- html_node(root_node, "table")
table_node

Read the table node as a data frame using `html_table` function


In [124]:
# Read the table node and convert it into a data frame, and print the data frame for review
df <- html_table(table_node)
df

## TASK 3: Pre-process and export the extracted data frame

The goal of task 3 is to pre-process the extracted data frame from the previous step, and export it as a csv file


Let's get a summary of the data frame.

In [125]:
# Print the summary of the data frame
summary(df)

As you can see from the summary, the column names are little bit different to understand and some column data types are not correct. For example, the `Tested` column shows as `character`.

As such, the data frame read from HTML table will need some pre-processing such as removing irrelvant columns, renaming columns, and convert columns into proper data types.


We have prepared a pre-processing function for you to conver the data frame but you can also try to write one by yourself


In [148]:
preprocess_covid_data_frame <- function(data_frame) {
    
    shape <- dim(data_frame)

    # Remove the World row
    data_frame <- data_frame[!(data_frame$`Country or region`=="World"),]
    # Remove the last row
    data_frame <- data_frame[1:172, ]
    
    # We dont need the Units and Ref columns, so can be removed
    data_frame["Ref."] <- NULL
    data_frame["Units[b]"] <- NULL
    
    # Renaming the columns
    names(data_frame) <- c("country", "date", "tested", "confirmed", "confirmed.tested.ratio", "tested.population.ratio", "confirmed.population.ratio")
    
    # Convert column data types
    data_frame$country <- as.factor(data_frame$country)
    data_frame$date <- as.factor(data_frame$date)
    data_frame$tested <- as.numeric(gsub(",","",data_frame$tested))
    data_frame$confirmed <- as.numeric(gsub(",","",data_frame$confirmed))
    data_frame$'confirmed.tested.ratio' <- as.numeric(gsub(",","",data_frame$`confirmed.tested.ratio`))
    data_frame$'tested.population.ratio' <- as.numeric(gsub(",","",data_frame$`tested.population.ratio`))
    data_frame$'confirmed.population.ratio' <- as.numeric(gsub(",","",data_frame$`confirmed.population.ratio`))
    
    return(data_frame)
}

Call the `preprocess_covid_data_frame` function


In [127]:
# call `preprocess_covid_data_frame` function and assign it to a new data frame
df <- preprocess_covid_data_frame(df)
df

Get the summary of the processed data frame again


In [128]:
# Print the summary of the processed data frame again
summary(df)

After pre-processing, you can see the columns and columns names are simplified, and column types are converted into correct types.


The data frame has following columns:

*   **country** - The name of the country
*   **date** - Reported date
*   **tested** - Total tested cases by the reported date
*   **confirmed** - Total confirmed cases by the reported date
*   **confirmed.tested.ratio** - The ratio of confirmed cases to the tested cases
*   **tested.population.ratio** - The ratio of tested cases to the population of the country
*   **confirmed.population.ratio** - The ratio of confirmed cases to the population of the country


OK, we can call `write.csv()` function to save the csv file into a file.


In [129]:
# Export the data frame to a csv file

write.csv(df, file = "covid19.csv", row.names = FALSE)

Note for IBM Watson Studio, there is no traditional "hard disk" associated with a R workspace.

Even if you call `write.csv()` method to save the data frame as a csv file, it won't be shown in IBM Cloud Object Storage asset UI automatically.

However, you may still check if the `covid.csv` exists using following code snippet:


In [130]:
# Get working directory
wd <- getwd()
# Get exported 
file_path <- paste(wd, sep="", "/covid.csv")
# File path
print(file_path)
file.exists(file_path)

**Optional Step**: If you have difficulties finishing above webscraping tasks, you may still continue with next tasks by downloading a provided csv file from here:


In [131]:
## Download a sample csv file
# covid_csv_file <- download.file("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/dataset/covid.csv", destfile="covid.csv")
# covid_data_frame_csv <- read.csv("covid.csv", header=TRUE, sep=",")

## TASK 4: Get a subset of the extracted data frame

The goal of task 4 is to get the 5th to 10th rows from the data frame with only `country` and `confirmed` columns selected


In [132]:
head(df, n = 10)

In [133]:
# OPTIONAL: Read covid_data_frame_csv from the csv file

# Get the 5th to 10th rows, with two "country" "confirmed" columns
df[5:10, c(1, 4)]

## TASK 5: Calculate worldwide COVID testing positive ratio

The goal of task 5 is to get the total confirmed and tested cases worldwide, and try to figure the overall positive ratio using `confirmed cases / tested cases`


In [134]:
head(df)

In [149]:
# Get the total confirmed cases worldwide
worldwide_cases <- sum(df$confirmed)
print("Worldwide positive cases: ")
worldwide_cases

# Get the total tested cases worldwide
worldwide_tests <- sum(df$tested)
print("Worldwide tests: ")
worldwide_tests

# Get the positive ratio (confirmed / tested)
positive_ratio_worldwide <- round(worldwide_cases / worldwide_tests, digit = 2)
print("Worldwide positive ratio: ")
positive_ratio_worldwide

## TASK 6: Get a country list which reported their testing data

The goal of task 6 is to get a catalog or sorted list of countries who have reported their COVID-19 testing data


In [136]:
# Get the `country` column
df$country

# Check its class (should be Factor)
class(df$country)

# Convert the country column into character so that you can easily sort them
df$country <- as.character(df$country)
class(df$country)

# Sort the countries AtoZ
sort(df$country)

# Sort the countries ZtoA
z_to_a_countries <- sort(df$country, decreasing = TRUE)

# Print the sorted ZtoA list
print(z_to_a_countries)

## TASK 7: Identify countries names with a specific pattern

The goal of task 7 is using a regular expression to find any countires starting with `United`


In [137]:
# OPTIONAL: Get the positions of countries starting with 'United'
row_nums <- grep("United.+", df$country)
print("Row numbers of countries starting with 'United'")
row_num

# Use a regular expression `United.+` to find matches
matches <- regexpr("United.+", df$country)

# Print the matched country names
regmatches(df$country, matches)

## TASK 8: Pick two countries you are interested, and then review their testing data

The goal of task 8 is to compare the COVID-19 test data between two countires, you will need to select two rows from the dataframe, and select `country`, `confirmed`, `confirmed-population-ratio` columns


In [138]:
tail(df)

In [139]:
# Select a subset (should be only one row) of data frame based on a selected country name and columns
zimbabwe_data <- df[172, c("country", "tested", "confirmed", "confirmed.population.ratio")]
zimbabwe_data

# Select a subset (should be only one row) of data frame based on a selected country name and columns
zambia_data <- df[171, c("country", "tested", "confirmed", "confirmed.population.ratio")]
zambia_data

In [140]:
diffirence_positive <- zimbabwe_data$confirmed - zambia_data$confirmed
diffirence_positive

In [141]:
difference_tested <- zimbabwe_data$tested - zambia_data$tested
difference_tested

As seen from the results Zambia has more positive cases. But Zimbabwe also has less tested people. 

## TASK 9: Compare which one of the selected countries has a larger ratio of confirmed cases to population

The goal of task 9 is to find out which country you have selected before has larger ratio of confirmed cases to population, which may indicate that country has higher COVID-19 infection risk


In [150]:
# Use if-else statement
if (zimbabwe_data$confirmed.population.ratio > zambia_data$confirmed.population.ratio) {
    print("Zimbabwe has a higher infection risk.")
 } else {
    print("Zambia has a higher infection risk.")
 }

## TASK 10: Find countries with confirmed to population ratio rate less than a threshold

The goal of task 10 is to find out which countries have the confirmed to population ratio less than 1%, it may indicate the risk of those countries are relatively low


In [143]:
head(df)

In [151]:
# Get a subset of any countries with `confirmed.population.ratio` less than the threshold
filtered_df <- filter(df, df$confirmed.population.ratio < 1)
filtered_df